In [1]:
import datetime
import time
import random
import requests
import json
import bson
import gzip
import os

LOCAL = 'http://localhost:8000/'
REMOTE = 'https://api-recintos.herokuapp.com/'
HOME = LOCAL
CARGA = 100

In [2]:
letras = 'ABCDEFGHIJKLMNOPQRSTUVXZ'
numeros = ''.join([str(i) for i in range(10)])
textos = 'ABCDEFGHIJKLMNOPQRSTUVXZ          abcdefghijklmnopqrstuvwxyz'
def random_str(num=3, fila=letras):
    result = ''
    for i in range(num):
        result += random.choice(fila)
    return result

placas = [random_str(3) + random_str(5, numeros) for i in range(100)]
reboques = [random_str(3) + random_str(5, numeros) for i in range(200)]
conteineres = [random_str(4) + random_str(7, numeros) for i in range(200)]
operadores = [random_str(11) for i in range(10)]
motoristas = [random_str(11) for i in range(50)]
textos = [random_str(random.randint(10, 20), textos) for i in range(50)]

In [3]:
pesagens = []
for r in range(CARGA):
    data = datetime.datetime.now().isoformat()
    operador = random.choice(operadores)
    conteiner = random.choice(conteineres)
    tara = random.randint(9000, 12000)
    pesobrutodeclarado = random.randint(3000, 15000)
    pesobalanca = tara + random.randint(-2000, 2000)
    placa = random.choice(placas)
    reboque = random.choice(reboques)
    texto = random.choice(textos)
    pesagem = \
    {'IDEvento': r,
     'capturaautomatica': True,
     'conteiner': random.choice(conteineres),
     'dataevento': data,
     'dataregistro': data,
     'retificador': False,
     'documentotransporte': texto,
     'operadorevento': operador,
     'operadorregistro': operador,
     'pesobalanca': pesobalanca,
     'pesobrutodeclarado': pesobrutodeclarado,
     'placa': placa,
     'placasemireboque': reboque,
     'taraconjunto': tara,
     'tipodocumentotransporte': 'CE'}
    pesagens.append(pesagem)


In [4]:
pesagens[:2]

[{'IDEvento': 0,
  'capturaautomatica': True,
  'conteiner': 'VKRA8921261',
  'dataevento': '2019-05-30T07:51:34.779306',
  'dataregistro': '2019-05-30T07:51:34.779306',
  'documentotransporte': 'ROmJTV bLP',
  'operadorevento': 'BONDCFLCEXS',
  'operadorregistro': 'BONDCFLCEXS',
  'pesobalanca': 10455,
  'pesobrutodeclarado': 11420,
  'placa': 'EAG51682',
  'placasemireboque': 'CKE57417',
  'retificador': False,
  'taraconjunto': 11778,
  'tipodocumentotransporte': 'CE'},
 {'IDEvento': 1,
  'capturaautomatica': True,
  'conteiner': 'JKKP4215489',
  'dataevento': '2019-05-30T07:51:34.779341',
  'dataregistro': '2019-05-30T07:51:34.779341',
  'documentotransporte': 'KinTefe cc K ',
  'operadorevento': 'MMHSTFCBGME',
  'operadorregistro': 'MMHSTFCBGME',
  'pesobalanca': 12068,
  'pesobrutodeclarado': 4773,
  'placa': 'PVU04827',
  'placasemireboque': 'BCG31959',
  'retificador': False,
  'taraconjunto': 11677,
  'tipodocumentotransporte': 'CE'}]

In [5]:
s0 = time.time()
for pesagem in pesagens[:CARGA // 2]:
    rv = requests.post(HOME + 'pesagemmaritimo',
                   json=pesagem)
    print(rv.status_code)
    print(rv.text)
s1 = time.time()

405
"Evento repetido ou campo invalido: (sqlite3.IntegrityError) UNIQUE constraint failed: pesagensmaritimo.recinto, pesagensmaritimo.IDEvento\n[SQL: INSERT INTO pesagensmaritimo (\"IDEvento\", dataevento, operadorevento, dataregistro, operadorregistro, recinto, \"request_IP\", retificador, documentotransporte, tipodocumentotransporte, numero, placa, placasemireboque, pesobrutodeclarado, taraconjunto, pesobalanca, capturaautomatica) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]\n[parameters: (0, '2019-05-30 07:51:34.779306', 'BONDCFLCEXS', '2019-05-30 07:51:34.779306', 'BONDCFLCEXS', '00001', '127.0.0.1', 0, 'ROmJTV bLP', 'CE', None, 'EAG51682', 'CKE57417', 11420, 11778, 10455, 1)]\n(Background on this error at: http://sqlalche.me/e/gkpj)"

201
6673958348723814915

201
3059081874479920381

201
4889468525994399638

201
-4775797251905336968

201
-8286320159321602457

201
-2497428813723666546

201
-7296405685153747197

201
320628054715715819

201
9056954035308279534

201
424

In [6]:
print('Gravados %d registros em %d segundos' % (CARGA // 2, s1-s0) )
print((s1 - s0) / (CARGA // 2))

Gravados 50 registros em 1 segundos
0.034726982116699216


In [7]:
s0 = time.time()
lista_eventos = []
for IDEvento in range(CARGA):
    rv = requests.get(HOME + 'pesagemmaritimo/' + str(IDEvento))
    if rv.status_code == 200:
        lista_eventos.append((rv.status_code, 
                              rv.json()))
#    print(rv.json())
s1 = time.time()

In [8]:
print('Consultados %d registros em %s segundos' % (len(lista_eventos), s1-s0))
print((s1 - s0) / len(lista_eventos))

Consultados 50 registros em 0.3716549873352051 segundos
0.007433099746704102


In [9]:
lista_eventos[:2]

[(200,
  {'ID': 1,
   'IDEvento': 0,
   'capturaautomatica': True,
   'dataevento': '2019-05-28T11:45:43.499000Z',
   'dataregistro': '2019-05-28T11:45:43.499000Z',
   'documentotransporte': 'string',
   'hash': -6711484747788190017,
   'numero': 'string',
   'operadorevento': 'string',
   'operadorregistro': 'string',
   'pesobalanca': 0,
   'pesobrutodeclarado': 0,
   'placa': 'string',
   'placasemireboque': 'string',
   'recinto': '00001',
   'request_IP': '127.0.0.1',
   'retificador': False,
   'taraconjunto': 0,
   'time_created': '2019-05-30T03:43:04Z',
   'tipodocumentotransporte': 'CE'}),
 (200,
  {'ID': 2,
   'IDEvento': 1,
   'capturaautomatica': True,
   'dataevento': '2019-05-30T07:51:34.779341Z',
   'dataregistro': '2019-05-30T07:51:34.779341Z',
   'documentotransporte': 'KinTefe cc K ',
   'hash': 6673958348723814915,
   'numero': None,
   'operadorevento': 'MMHSTFCBGME',
   'operadorregistro': 'MMHSTFCBGME',
   'pesobalanca': 12068,
   'pesobrutodeclarado': 4773,
   'p

## Teste de tamanho de arquivos em formato json, json .gzip, bson e bson .gzip

In [10]:
def tofile(lista, newfilename, zipped=False):
    """Salva instância em arquivo (padrão BSON)."""
    with open(newfilename, 'wb') as f:
        payload = bson.BSON.encode(lista)
        if zipped:
            payload = gzip.compress(payload)
        f.write(payload)

json_pesagens = {'PesagemMaritimo': pesagens}

with open('test.json', 'w', encoding='utf-8', newline='') as json_out:
    json.dump(json_pesagens, json_out)
size_json = os.path.getsize('test.json')

with open('test.json.gzip', 'wb') as json_out:
    payload = json.dumps(json_pesagens)
    payload = gzip.compress(payload.encode('utf-8'))
    json_out.write(payload)
size_jsongzip = os.path.getsize('test.json.gzip')

tofile(json_pesagens, 'test.bson')
size_bson = os.path.getsize('test.bson')

tofile(json_pesagens, 'test.bson.gzip', zipped=True)
size_bsongzip = os.path.getsize('test.bson.gzip')


In [26]:
print('size_json: %d, %d%s' % (size_json, (size_json/size_json)*100, '%'))
print('size_json_gzip: %d, %d%s ' % (size_jsongzip, (size_jsongzip/size_json)*100, '%'))
print('size_bson: %d, %d%s ' % (size_bson, (size_bson/size_json)*100, '%'))
print('size_bson_gzip: %d, %d%s ' % (size_bsongzip, (size_bsongzip/size_json)*100, '%'))

size_json: 45428, 100%
size_json_gzip: 5848, 12% 
size_bson: 42157, 92% 
size_bson_gzip: 6244, 13% 


In [27]:
s0 = time.time()
query = {'IDEvento': 0,
       'tipoevento': 'PesagemMaritimo'}
r = requests.get(HOME + 'get_eventos_novos',
                  data=query,
                  headers=headers)
s1 = time.time()
print(r.status_code)
print(r.text[:1000])

200
[
  {
    "ID": 2, 
    "IDEvento": 1, 
    "capturaautomatica": true, 
    "dataevento": "2019-05-28T16:50:02.450497Z", 
    "dataregistro": "2019-05-28T16:50:02.450497Z", 
    "documentotransporte": "ScQP s  GCS Q", 
    "hash": -6228561499361064826, 
    "numero": null, 
    "operadorevento": "IBBOSGPFVOL", 
    "operadorregistro": "IBBOSGPFVOL", 
    "pesobalanca": 9063, 
    "pesobrutodeclarado": 6158, 
    "placa": "CZB55249", 
    "placasemireboque": "IQA81890", 
    "recinto": "00001", 
    "request_IP": "127.0.0.1", 
    "retificador": false, 
    "taraconjunto": 10913, 
    "time_created": "2019-05-28T19:50:04Z", 
    "tipodocumentotransporte": "CE"
  }, 
  {
    "ID": 3, 
    "IDEvento": 2, 
    "capturaautomatica": true, 
    "dataevento": "2019-05-28T16:50:02.450554Z", 
    "dataregistro": "2019-05-28T16:50:02.450554Z", 
    "documentotransporte": "OdJJJZNBcjXlBRzGHuS", 
    "hash": 7426620730026009243, 
    "numero": null, 
    "operadorevento": "IERCPJDGRUS", 
    "o

In [28]:
lista_eventos = json.loads(r.text)

In [29]:
print('Consultados %d registros em %s segundos' % (len(lista_eventos), s1-s0))
print((s1 - s0) / len(lista_eventos))

Consultados 49 registros em 0.014060497283935547 segundos
0.00028694892416194996


In [30]:
lista_eventos[:10]

[{'ID': 2,
  'IDEvento': 1,
  'capturaautomatica': True,
  'dataevento': '2019-05-28T16:50:02.450497Z',
  'dataregistro': '2019-05-28T16:50:02.450497Z',
  'documentotransporte': 'ScQP s  GCS Q',
  'hash': -6228561499361064826,
  'numero': None,
  'operadorevento': 'IBBOSGPFVOL',
  'operadorregistro': 'IBBOSGPFVOL',
  'pesobalanca': 9063,
  'pesobrutodeclarado': 6158,
  'placa': 'CZB55249',
  'placasemireboque': 'IQA81890',
  'recinto': '00001',
  'request_IP': '127.0.0.1',
  'retificador': False,
  'taraconjunto': 10913,
  'time_created': '2019-05-28T19:50:04Z',
  'tipodocumentotransporte': 'CE'},
 {'ID': 3,
  'IDEvento': 2,
  'capturaautomatica': True,
  'dataevento': '2019-05-28T16:50:02.450554Z',
  'dataregistro': '2019-05-28T16:50:02.450554Z',
  'documentotransporte': 'OdJJJZNBcjXlBRzGHuS',
  'hash': 7426620730026009243,
  'numero': None,
  'operadorevento': 'IERCPJDGRUS',
  'operadorregistro': 'IERCPJDGRUS',
  'pesobalanca': 11539,
  'pesobrutodeclarado': 7916,
  'placa': 'KPH7378

In [31]:
s0 = time.time()
files = {'file': ('test.json', open('test.json', 'r'), 'text/json')}
headers = {}
# headers['Content-Type'] = 'image/jpeg'
r = requests.post(HOME + 'set_eventos_novos',
                  data=data,
                  files=files,
                  headers=headers)
s1 = time.time()
print(r.status_code)
print(r.text)


201
[
  {
    "IDEvento": 50, 
    "hash": 6901618861254910335
  }, 
  {
    "IDEvento": 51, 
    "hash": 5541878240847989619
  }, 
  {
    "IDEvento": 52, 
    "hash": 6831252833105351224
  }, 
  {
    "IDEvento": 53, 
    "hash": -3757962496483993345
  }, 
  {
    "IDEvento": 54, 
    "hash": 1354141284648884911
  }, 
  {
    "IDEvento": 55, 
    "hash": -556991403825226245
  }, 
  {
    "IDEvento": 56, 
    "hash": 8028122056083013573
  }, 
  {
    "IDEvento": 57, 
    "hash": 5449133351729161668
  }, 
  {
    "IDEvento": 58, 
    "hash": 4268843537034976252
  }, 
  {
    "IDEvento": 59, 
    "hash": -6718992851902010518
  }, 
  {
    "IDEvento": 60, 
    "hash": -6361486284902505758
  }, 
  {
    "IDEvento": 61, 
    "hash": 4919737457380698160
  }, 
  {
    "IDEvento": 62, 
    "hash": -1639249394911906483
  }, 
  {
    "IDEvento": 63, 
    "hash": 5671522007877914772
  }, 
  {
    "IDEvento": 64, 
    "hash": -1461659623945634662
  }, 
  {
    "IDEvento": 65, 
    "hash": 7495263

In [32]:
print('Gravados %d registros em %s segundos' % (CARGA // 2, s1-s0))
print((s1 - s0) / (CARGA // 2))

Gravados 50 registros em 0.1287364959716797 segundos
0.002574729919433594
